# Recommender system

FILTER BY CONSTRAINTS (kcal, cuisine, time, vegan, ...)  
CREATE MEALS  
FILTRER LES USERS ET LES RECETTES EN FONCTION DU NOMBRE DE SAVE  
=> PUIS ENREGISTRER LES BAILS QUI SERVENT DANS DES FICHIERS !!!!!

DESCRIPTION

In [1]:
import pandas as pd
import numpy as np

In [123]:
saved_recipes = pd.read_json('saved_recipes.json')
saved_recipes = saved_recipes[['user', 'recipe_id']]
saved_recipes.head(10)

,user,recipe_id
0,SapnaPadte,ROASTED-BEET_-PEAR-_-WALNUT-SALAD-1895162
1,SapnaPadte,Farro-Salad-with-Roasted-Mushrooms-and-Parmesa...
2,SapnaPadte,Braised-Short-Ribs-2019358
3,SapnaPadte,Mushroom-Risotto-1571924
4,SapnaPadte,Crockpot-Chicken-and-Stuffing-1900892
5,SapnaPadte,Swiss-Chard-And-Sweet-Potato-Gratin-1818910
6,SapnaPadte,Glazed-Tofu-with-Bok-Choy-1459225
7,SapnaPadte,Ricotta-and-Oyster-Mushroom-Lasagna-1159281
8,SapnaPadte,Cilantro-Lime-Coconut-Rice-1086839
9,SapnaPadte,Garlic-Roasted-Asparagus-1053963


In [124]:
all_recipes = list(saved_recipes.recipe_id.unique())
all_users = list(saved_recipes.user.unique())
print('We have data from {} users who saved in total {} recipes '.format(len(all_users), len(saved_recipes)) +
      'making an average of {} recipes saved per user '.format(round(len(saved_recipes)/len(all_users), 2)))

We have data from 4046 users who saved in total 109264 recipes making an average of 27 recipes saved per user 


For better performance in terms of speed and results, we only keep users who saved at least 10 recipes and recipes which have been saved at least 3 times to reduce the size of the dataset in order to keep only items of interest

In [125]:
recipes_cp = all_recipes.copy()
for r in recipes_cp:
    n_users_who_saved_recipe = len(saved_recipes[saved_recipes.recipe_id == r])
    if(n_users_who_saved_recipe < 3):
        all_recipes.remove(r)
users_cp = all_users.copy()
for u in users_cp:
    n_saved_recipes_for_user = len(saved_recipes[saved_recipes.user == u])
    if(n_saved_recipes_for_user < 20):
        all_users.remove(u)
print('From this, it now remains {} users and {} recipes'.format(len(all_users), len(all_recipes)))

From this, it now remains 3261 users and 6642 recipes


In [415]:
pd.DataFrame(all_recipes).to_csv('all_recipes.csv')

We save this dataframe to use it later on without having to do all the computations.

In [126]:
to_drop = []
for i in range(len(saved_recipes)):
    u = saved_recipes.loc[i].user
    r = saved_recipes.loc[i].recipe_id
    if(not u in all_users or not r in all_recipes):
        to_drop.append(i)
        
to_save = saved_recipes.drop(to_drop)
to_save.reset_index()
to_save.to_csv('useful_saved_recipes.csv')

In [283]:
saved_recipes = pd.read_csv('useful_saved_recipes.csv')[['user', 'recipe_id']]
all_recipes = list(saved_recipes.recipe_id.unique())
all_users = list(saved_recipes.user.unique())

Now we transform this data into a dataframe one containing 0 or 1 depending on if the user has saved or not the recipe.

In [285]:
saved_recipes_for_users = dict()
for u in all_users:
    saved_recipes_for_users[u] = list(saved_recipes[saved_recipes.user == u].recipe_id)

In [286]:
matrix_data = np.zeros((len(all_users), len(all_recipes)))
for i in range(len(all_users)):
    u = all_users[i]
    for j in range(len(all_recipes)):
        r = all_recipes[j]
        if(r in saved_recipes_for_users[u]):
            matrix_data[i, j] = 1
        

In [287]:
saved = pd.DataFrame(matrix_data, index = all_users, columns = all_recipes)

To save or reload the data matrix

In [288]:
saved.to_csv('saved_matrix.csv')

In [289]:
saved = pd.read_csv('saved_matrix.csv', index_col='Unnamed: 0')

Now we can make our reommendations based on the saved recipes. 

In [290]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

# reload the data
saved = pd.read_csv('saved_matrix.csv', index_col='Unnamed: 0')

# As a first step we normalize the user vectors to unit vectors.
magnitude = np.sqrt(saved.sum(axis=1)) #np.sqrt(np.square(saved).sum(axis=1)) but 0^2=0 and 1^2=1
saved = saved.divide(magnitude, axis='index')

# remove nan values
saved[np.isnan(saved)] = 0

def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index=data_items.columns, columns=data_items.columns)
    return sim

We first make item-item recommendations

In [291]:
# Build the similarity matrix
data_matrix = calculate_similarity(saved)

print('recipes most similar to Mushroom-Risotto-1571924: \n')
print(data_matrix.loc['Mushroom-Risotto-1571924'].nlargest(5))

recipes most similar to Mushroom-Risotto-1571924: 

Mushroom-Risotto-1571924                                   1.000000
Farro-Salad-with-Roasted-Mushrooms-and-Parmesan-1542254    0.339569
POTATO-GNOCCHI-WITH-MUSHROOMS-AND-FETA-1422882             0.289354
Bacon-Cheddar-Scallion-Scones-677696                       0.283927
Greek-Moussaka-2119898                                     0.267662
Name: Mushroom-Risotto-1571924, dtype: float64


Finally we compute user-item recommendation

In [292]:
user = 'Axel' # my user ID that I created for demo purposes

def recommend_for_user(user):
    
    # Users likes for all items as a sparse vector.
    user_rating_vector = saved.loc[user]
    
    # Get the recipes the user has liked.
    known_user_likes = user_rating_vector[user_rating_vector > 0].index.values

    # Calculate the score.
    score = data_matrix.dot(user_rating_vector).div(data_matrix.sum(axis=1))

    # Remove the known likes from the recommendation.
    score = score.drop(known_user_likes)    
    
    return known_user_likes, score

# Print the known saved and the top 10 recommendations.
known_user_likes, score = recommend_for_user(user)
print('Saved items by user {}:'.format(user))
print(known_user_likes)
print()
print('Recommendations for user {}:'.format(user))
print(score.nlargest(10))

Saved items by user Axel:
['Spaghetti-alla-Carbonara-902641' 'Classic-Swiss-Fondue-My-Recipes'
 'Lemon-Madeleines-Martha-Stewart-195085'
 'Tarte-Tatin-An-Easy-Classic-French-Dessert-141...'
 'Cherry-Clafouti-513493' 'Chocolate-mousse_-french-style-357262'
 'French-Baguette-Sandwich-374433' 'Croque-Monsieur-Martha-Stewart-227846'
 'Creme-Fraiche-Chicken-Allrecipes'
 'Delicious-Grilled-Hamburgers-Allrecipes'
 'Chicken_-Spinach-and-Artichoke-Calzones-1019821' 'My-Tiramisu-571297'
 'Greek-Salad-1398313' 'Beef-taco-skillet-348744'
 'paghetti-and-Meatballs-Martha-Stewart'
 'Quick-Barbecue-Chicken-My-Recipes' 'Sweet-Potato-Fries-Martha-Stewart'
 'Homemade-Guacamole-1058667' 'Four-Cheese-Pizza-MyRecipes-245856'
 'Quiche-Lorraine-Ii-Allrecipes' 'French-Crepes-1258186']

Recommendations for user Axel:
Mandu-Recipe-_Korean-Dumplings_-496808                        0.007605
Pan-Seared-Ahi-Tuna-1192074                                   0.006090
Pasta-with-Braised-Broccoli-and-Tomato-Serious-Eats-200

# Interactive recommender

In [293]:
from ipywidgets import widgets

Now, We will make the recommendations based on given constraints chosen by the users. These constraints can be nutrition, time, course or cuisine type constraints. For this we will use two dataframes. The first one contains the recipes which will help us to make recommendations based on cuisine, time, or even ingredients constraints. The last one contains nutrition informations which will also help us to make recommendations based this time on nutrional constraints.

In [294]:
recipes = pd.read_csv('recipes.csv')
recipes.head()

,recipe_id,name,totalTime,course,cuisine,ingredients
0,Mushroom-risotto-352492,Mushroom Risotto,1800.0,Side Dishes,Italian,"['2 cups Baby Bella mushrooms, sliced', '2 cup..."
1,Filipino-bbq-pork-skewers-352163,Filipino BBQ Pork Skewers,2400.0,Main Dishes,Barbecue,"['2.5 lb pork country style ribs, all fat trim..."
2,Mushroom-and-Roasted-Garlic-Risotto-525026,Mushroom and Roasted Garlic Risotto,5100.0,Main Dishes,Italian,"['2 whole garlic heads', '2 tablespoons plus 2..."
3,Gratin-Dauphinois-_scalloped-Potatoes-With-Che...,Gratin Dauphinois (Scalloped Potatoes with Che...,3300.0,Side Dishes,French,"['1 garlic clove, halved', 'Cooking spray', '6..."
4,Delicious-Grilled-Hamburgers-Allrecipes,Delicious Grilled Hamburgers,900.0,Main Dishes,Barbecue,"['1 pound lean ground beef', '1 tablespoon Wor..."


In [430]:
nutrition = pd.read_csv('nutrition.csv')
filtered_nutrition = nutrition[['recipe_id', 'attribute', 'description', 'unit_name', 'unit_abbreviation', 'value']]
filtered_nutrition = filtered_nutrition.set_index(['recipe_id', 'attribute'])
filtered_nutrition.head(10)

description  \
recipe_id               attribute                                       
Mushroom-risotto-352492 FAT_KCAL                                  NaN   
                        FAPU       Fatty acids, total polyunsaturated   
                        CAMD5                             Campesterol   
                        FOLDFE                            Folate, DFE   
                        FOLFD                            Folate, food   
                        MN                              Manganese, Mn   
                        TOCPHA           Vitamin E (alpha-tocopherol)   
                        FAMS       Fatty acids, total monounsaturated   
                        VITC           Vitamin C, total ascorbic acid   
                        CARTB                          Carotene, beta   

                                  unit_name unit_abbreviation   value  
recipe_id               attribute                                      
Mushroom-risotto-352492 FAT_KCAL    calorie              kcal   80.00  
                        FAPU           gram                 g    0.24  
                        CAMD5          gram                 g    0.00  
                        FOLDFE      mcg_DFE           mcg_DFE  298.07  
                        FOLFD          gram                 g    0.00  
                        MN             gram                 g    0.00  
                        TOCPHA         gram                 g    0.00  
                        FAMS           gram                 g    4.79  
                        VITC           gram                 g    0.01  
                        CARTB          gram                 g    0.00

UTILISER ENERC_KCAL

In [296]:
# keep only the kcal column
cal_df = nutrition
cal_df = cal_df[cal_df.attribute == 'FAT_KCAL'].reset_index()
cal_df = cal_df[['recipe_id', 'value']]
cal_df.columns = ['recipe_id', 'kcal']
cal_df.head()

,recipe_id,kcal
0,Mushroom-risotto-352492,80.0
1,Filipino-bbq-pork-skewers-352163,320.0
2,Mushroom-and-Roasted-Garlic-Risotto-525026,540.0
3,Gratin-Dauphinois-_scalloped-Potatoes-With-Che...,50.0
4,Delicious-Grilled-Hamburgers-Allrecipes,110.0


In [297]:
df_original = pd.merge(recipes, cal_df, how='outer', on='recipe_id')
df_original = df_original[['recipe_id', 'name', 'totalTime', 'course', 'cuisine', 'kcal', 'ingredients']]
df_original['totalTime'] = df_original['totalTime']/60
df_original.head()

,recipe_id,name,totalTime,course,cuisine,kcal,ingredients
0,Mushroom-risotto-352492,Mushroom Risotto,30.0,Side Dishes,Italian,80.0,"['2 cups Baby Bella mushrooms, sliced', '2 cup..."
1,Filipino-bbq-pork-skewers-352163,Filipino BBQ Pork Skewers,40.0,Main Dishes,Barbecue,320.0,"['2.5 lb pork country style ribs, all fat trim..."
2,Mushroom-and-Roasted-Garlic-Risotto-525026,Mushroom and Roasted Garlic Risotto,85.0,Main Dishes,Italian,540.0,"['2 whole garlic heads', '2 tablespoons plus 2..."
3,Gratin-Dauphinois-_scalloped-Potatoes-With-Che...,Gratin Dauphinois (Scalloped Potatoes with Che...,55.0,Side Dishes,French,50.0,"['1 garlic clove, halved', 'Cooking spray', '6..."
4,Delicious-Grilled-Hamburgers-Allrecipes,Delicious Grilled Hamburgers,15.0,Main Dishes,Barbecue,110.0,"['1 pound lean ground beef', '1 tablespoon Wor..."


In [437]:
def update_x_range(*args):
    if(x.value > y.value):
        x.value = y.value
    y.observe(update_x_range, 'value')
    
def update_y_range(*args):
    if(y.value < x.value):
        y.value = x.value
    x.observe(update_y_range, 'value')

def f(Filter_by_kcal, min_kcal, max_kcal, User, Course, Cuisine, Filter_by_time, t):
    df = df_original.copy()
    df['URL'] = 'https://www.yummly.com/#recipe/' + df['recipe_id']
    if(Filter_by_kcal):
        df = df[df.kcal >= min_kcal]
        df = df[df.kcal <= max_kcal]
    if(Course != 'All'):
        df = df[df.course==Course]
    if(Cuisine != 'All'):
        df = df[df.cuisine==Cuisine]      
    if(Filter_by_time):
        df = df[df.totalTime <= t]
    if(df.empty):
        print('Sorry, no recommendation')
    else:
        columns = ['name', 'kcal', 'totalTime']
        if(Course == 'All'):
            columns.append('course')
        if(Cuisine == 'All'):
            columns.append('cuisine')
        columns.append('URL')    

        known_user_likes, score = recommend_for_user(User)
        score_df = pd.DataFrame(score).reset_index()
        score_df.columns = ['recipe_id', 'score']
        df = pd.merge(score_df, df, how='inner', on='recipe_id') # how = outer to see all
        df = df.sort_values(by = 'score', ascending=False)
        
        #print('Here what user', User, 'liked:')
        #print(known_user_likes)
        #print()
        
        print('Here are some recommendations for user', User, ':')
        
        #return df[['URL']].head().style.format(make_clickable)
        return df[columns].set_index('name').head()

def make_clickable(link):
    begin = 'https://www.yummly.com/#recipe/'
    return '<a href="{}">{}</a>'.format(link, link[len(begin):])

In [438]:
x = widgets.IntSlider(min=0, max=2000, step=1, value=0, description='Minimum number of kcal')
y = widgets.IntSlider(min=0, max=2000, step=1, value=2000, description='Maximum number of kcal')
cuisines = sorted(df_original.cuisine.unique())
cuisines.insert(0, 'All')
courses = sorted(df_original.course.unique())
courses.insert(0, 'All')
t = widgets.IntSlider(min=0, max=120, step=1, value=60, description='Maximum time')
user_select = users[:10];
user_select.insert(0, 'Axel')

bite = widgets.interact(f, Filter_by_kcal=True, min_kcal=x, max_kcal=y, 
                 User=user_select, Cuisine=cuisines, Course=courses,
                 Filter_by_time=True, t=t);
update_x_range()
update_y_range()

Here are some recommendations for user Axel :


,kcal,totalTime,course,cuisine,URL
name,,,,,
Thai Stir Fried Noodles (Pad See Ew),130.0,15.0,Main Dishes,Asian,https://www.yummly.com/#recipe/Thai-Stir-Fried...
"Penne Pasta with Spicy Italian Sausage, Mushrooms, and Spinach",150.0,30.0,Main Dishes,Italian,https://www.yummly.com/#recipe/Penne-pasta-wit...
Easy Lo Mein,50.0,25.0,Main Dishes,Asian,https://www.yummly.com/#recipe/Easy-Lo-Mein-75...
Mexican Lasagna,120.0,50.0,Main Dishes,Southwestern,https://www.yummly.com/#recipe/Mexican-lasagna...
Italian Sausage and Pasta with Tomato Cream Sauce,230.0,25.0,Main Dishes,Italian,https://www.yummly.com/#recipe/Italian-Sausage...


- un click voir d'autres recommandations 
- mettre les users dans le menu qui ont le plus de saved
- interface ou on choisit un user, affiche ce qu'il aime, et des bails recommandés en fonction des contraintes  
=> trier les recettes restantes pour un utilisateur donné dans l'ordre de ses recommandations   
- demander une liste d'ingrédients et montrer ce qu'on peut faire avec 
- LES KCAL SONT PAS LES BONNES !!!
- Attention aux duplicats !
- Faire marcher les URL !

Il faut récuperer tous les plats qu'on utilise et les avoir dans recipes etc aussi pour que ca marche